In [10]:
def read_binary(file_path):
    import struct
    import math
    import pandas as pd

    # Инициализация списков для основных полей
    power = []
    age = []
    coordinate_x = []
    coordinate_y = []
    angle_tetta = []
    angle_phi = []
    energy = []
    time = []
    
    # Дополнительные важные параметры
    primary_particle = []  # PART0 (14 - p, 5626 - Fe)
    primary_energy = []    # E0 в ГэВ
    n_muons = []          # NMU (число мюонов в ШАЛ)
    n_hadrons = []        # NHADR (число адронов в ШАЛ)
    first_interaction_height = []  # H1INT (высота 1 взаимодействия в см)

    with open(file_path, 'rb') as binary_file:
        while True:
            try:
                # Пропускаем первые 3 поля (N_event, NRUN, NEVENT)
                binary_file.read(4 * 3)
                
                # Читаем тип первичной частицы (14 - p, 5626 - Fe)
                part0 = struct.unpack('f', binary_file.read(4))[0]
                primary_particle.append(part0)
                
                # Читаем энергию первичной частицы (ГэВ)
                e0 = struct.unpack('f', binary_file.read(4))[0]
                primary_energy.append(e0)
                
                # Чтение зенитного угла
                tetta = struct.unpack('f', binary_file.read(4))[0]
                angle_tetta.append(tetta)
                
                # Чтение азимутального угла
                phi = struct.unpack('f', binary_file.read(4))[0]
                angle_phi.append(phi)
                
                # Чтение координат оси ШАЛ
                x0 = struct.unpack('f', binary_file.read(4))[0]
                coordinate_x.append(x0)
                
                y0 = struct.unpack('f', binary_file.read(4))[0]
                coordinate_y.append(y0)
                
                # Чтение высоты первого взаимодействия
                h1int = struct.unpack('f', binary_file.read(4))[0]
                first_interaction_height.append(h1int)
                
                # Пропускаем NGAM, NEL
                binary_file.read(4 * 2)
                
                # Чтение числа адронов
                nhadr = struct.unpack('f', binary_file.read(4))[0]
                n_hadrons.append(nhadr)
                
                # Чтение числа мюонов
                nmu = struct.unpack('f', binary_file.read(4))[0]
                n_muons.append(nmu)
                
                # Чтение параметров ШАЛ
                power_eas = struct.unpack('f', binary_file.read(4))[0]
                power.append(math.log10(power_eas))
                
                age_eas = struct.unpack('f', binary_file.read(4))[0]
                age.append(age_eas)
                
                # Пропускаем NVD_edep, NVD_npe, MuBundle, MuTrackLenNVD, nMuNVD, eMuNVD, eMuNVD1
                binary_file.read(4 * 7)
                
                # Пропускаем новые поля 2021 (23-34) и AmplKSM[7][4][4][6] (672 значения)
                binary_file.read(4 * (12 + 672))
                
                # Пропускаем новые поля 2021 (707-771)
                binary_file.read(4 * 64)
                
                # Пропускаем EdepCntSCT[9][5][2] (90 значений)
                binary_file.read(4 * 90)
                
                # Чтение EdepDetNE[9][4][4] (144 значения) - используем для energy
                edep_det_ne = struct.unpack('f'*144, binary_file.read(4*144))
                energy.append(edep_det_ne)
                
                # Чтение TimDetNE[9][4][4][4] (576 значений) - берем пороговые времена
                tim_det_ne = struct.unpack('f'*576, binary_file.read(4*576))
                threshold_time = tim_det_ne[::4]  # Берем каждое 4-е значение (max время)
                time.append(threshold_time)
                
                # Пропускаем EdepStNE[9][4] (36 значений) и TimStNE[9][4][4] (144 значения)
                binary_file.read(4 * (36 + 144))
                
                # Чтение маркера (1762)
                binary_file.read(4)
                

            except struct.error:
                # Достигнут конец файла
                break

    # Создание DataFrame
    df = pd.DataFrame({
        'power': power,
        'age': age,
        'x': coordinate_x,
        'y': coordinate_y,
        'tetta': angle_tetta,
        'phi': angle_phi,
        'energy': energy,
        'threshold_time': time,
        'primary_particle': primary_particle,
        'primary_energy': primary_energy,
        'n_muons': n_muons,
        'n_hadrons': n_hadrons,
        'first_interaction_height': first_interaction_height
    })

    return df


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import ast
from numpy import sin, cos
from IPython.display import display

In [11]:
# Загрузка и подготовка данных
def load_data(file_path):
    data = read_binary(file_path)
    # Разбор столбцов energy и threshold_time в отдельные DataFrame
    energy_df = pd.DataFrame(data['energy'].to_list(), columns=[f"energy_{i}" for i in range(144)])
    threshold_times_df = pd.DataFrame(data['threshold_time'].to_list(), columns=[f"threshold_time_{i}" for i in range(144)])
    
    # Объединение всех данных
    data = pd.concat([data, energy_df, threshold_times_df], axis=1)
    
    # Добавление sin и cos для угла phi
    data['sin'] = np.sin(data['phi'])
    data['cos'] = np.cos(data['phi'])
    
    # Удаление ненужных столбцов
    data = data.drop(columns=['energy', 'threshold_time', 'phi'], axis=1)
    
    # Разделение на признаки и целевую переменную
    X = data.drop(columns=['power', 'age', 'x', 'y', 'tetta', 'sin', 'cos'], axis=1)
    y = data[['power', 'age', 'x', 'y', 'tetta', 'sin', 'cos']]
    display(X.head())
    display(y.head())
    return X, y

In [5]:
# Нейронная сеть
class MultiTargetNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(MultiTargetNN, self).__init__()
        # Общее компактное представление признаков
        self.shared_encoder = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),

            # Бутылочное горлышко
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # Минимальные ветки для каждого таргета
        self.heads = nn.ModuleList([
            self._create_minimal_head() for _ in range(output_size)
        ])

    def _create_minimal_head(self):
        return nn.Sequential(
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        features = self.shared_encoder(x)
        outputs = [head(features) for head in self.heads]
        return torch.cat(outputs, dim=1)


In [5]:
import torch
import numpy as np
from torch.optim.lr_scheduler import ReduceLROnPlateau

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=100, patience=10, device='cuda'):
    # Перенос модели на устройство (GPU)
    model = model.to(device)

    # Инициализация шедулера
    scheduler = ReduceLROnPlateau(
        optimizer,
        mode='min',    # Минимизируем loss
        factor=0.5,    # Уменьшаем LR в 2 раза
        patience=5,    # Ждём 5 эпох без улучшений
        verbose=True,  # Выводим сообщения
        min_lr=1e-6    # Минимальный LR
    )

    best_val_loss = float('inf')
    patience_counter = 0
    history = {'train_loss': [], 'val_loss': [], 'lr': []}

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)

        train_loss /= len(train_loader.dataset)
        history['train_loss'].append(train_loss)

        model.eval()
        val_loss = 0.0
        all_preds = []
        all_targets = []

        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs = inputs.to(device)
                targets = targets.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)

                all_preds.append(outputs.cpu().numpy())
                all_targets.append(targets.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        history['val_loss'].append(val_loss)
        history['lr'].append(optimizer.param_groups[0]['lr'])

        # Обновление LR по шедулеру
        scheduler.step(val_loss)

        val_preds = np.concatenate(all_preds)
        val_targets = np.concatenate(all_targets)

        # Вычисление метрик
        print(f'\nEpoch {epoch+1}/{epochs}')
        print(f'LR: {optimizer.param_groups[0]["lr"]:.2e}')
        print(f'Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

        metrics = calculate_metrics(val_targets, val_preds, ['power', 'age', 'x', 'y', 'tetta', 'sin_phi', 'cos_phi'])
        for target, metric in metrics.items():
            print(f"{target}:")
            print(f"  MAE: {metric['MAE']:.4f}, MSE: {metric['MSE']:.4f}, RMSE: {metric['RMSE']:.4f}, R2: {metric['R2']:.4f}")

        # Ранняя остановка
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'Early stopping after {epoch+1} epochs')
                break

    # Загрузка лучшей модели
    model.load_state_dict(torch.load('best_model.pth', map_location=device))
    return model, history

In [6]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# Функция для вычисления метрик
def calculate_metrics(y_true, y_pred, target_names):
    metrics = {}
    for i, name in enumerate(target_names):
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        mse = mean_squared_error(y_true[:, i], y_pred[:, i])
        r2 = r2_score(y_true[:, i], y_pred[:, i])

        metrics[name] = {
            'MAE': mae,
            'MSE': mse,
            'RMSE': np.sqrt(mse),
            'R2': r2
        }
    return metrics

In [8]:
# Основной код
if __name__ == '__main__':
    # Проверка доступности GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

    # Загрузка данных
    file_path = '/kaggle/input/nuclear-it-hack/spe27p_100k_2022_correct.dat'
    features, targets = load_data(file_path)

    # Разделение на train и test
    X_train, X_test, y_train, y_test = train_test_split(
        features, targets, test_size=0.2, random_state=42
    )

    # Преобразование в тензоры PyTorch и перенос на GPU
    train_dataset = TensorDataset(
        torch.FloatTensor(X_train.values).to(device),
        torch.FloatTensor(y_train.values).to(device)
    )
    test_dataset = TensorDataset(
        torch.FloatTensor(X_test.values).to(device),
        torch.FloatTensor(y_test.values).to(device)
    )

    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    # Инициализация модели и перенос на GPU
    input_size = X_train.shape[1]
    output_size = y_train.shape[1]
    model = MultiTargetNN(input_size, output_size).to(device)

    # Функция потерь и оптимизатор
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.005, weight_decay=0.005)

    # Динамическое изменение LR
    scheduler = optim.lr_scheduler.CyclicLR(
      optimizer,
      base_lr=0.001,
      max_lr=0.01,
      step_size_up=500,
      cycle_momentum=False
    )

    # Обучение
    trained_model = train_model(
        model, train_loader, val_loader, criterion, optimizer, epochs=100, patience=20
    )

Using device: cuda


,primary_particle,primary_energy,n_muons,n_hadrons,first_interaction_height,energy_0,energy_1,energy_2,energy_3,energy_4,...,threshold_time_134,threshold_time_135,threshold_time_136,threshold_time_137,threshold_time_138,threshold_time_139,threshold_time_140,threshold_time_141,threshold_time_142,threshold_time_143
0,14.0,1024144.000,10006.0,815.0,2200590.750,0.000000,0.000000,0.101911,0.000000,10.222194,...,0.000000,91881.507812,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,14.0,1670368.625,14978.0,1854.0,1621342.125,0.000000,6.218760,0.000000,4.344400,0.000000,...,0.000000,0.000000,61217.339844,61166.957031,61161.582031,61212.566406,61184.625000,61236.789062,0.000000,0.00000
2,14.0,1141731.125,10802.0,725.0,1894253.375,0.000000,0.000000,0.000000,0.000000,0.344776,...,0.000000,0.000000,78678.609375,78690.390625,0.000000,0.000000,78712.031250,78717.500000,78714.734375,78708.06250
3,14.0,1102080.125,16449.0,3024.0,1576046.875,10.767822,8.262124,2.476740,13.453603,15.366792,...,52149.480469,52140.515625,52160.343750,0.000000,52162.656250,0.000000,52230.859375,52154.417969,52199.988281,52148.78125
4,14.0,1272719.750,10435.0,1372.0,2638823.500,10.822570,22.756126,28.512867,17.532497,12.286385,...,88170.554688,88115.945312,88077.148438,0.000000,88060.843750,88135.984375,88109.804688,88076.132812,88117.335938,0.00000


,power,age,x,y,tetta,sin,cos
0,4.166507,1.444716,13.572407,37.022316,37.367474,-0.188303,-0.982111
1,4.933104,1.430547,40.330677,-61.980999,28.912228,-0.703970,-0.710230
2,4.133581,1.459046,-1.046695,-63.925430,37.267426,-0.125979,0.992033
3,5.190620,1.326983,40.293152,-21.836197,3.205503,0.205486,0.978660
4,5.021614,1.336776,15.542248,10.037846,6.299241,0.959007,0.283384


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



Epoch 1/100
LR: 1.00e-03
Train Loss: 96.8366, Val Loss: 60.7797
power:
  MAE: 0.2304, MSE: 0.1004, RMSE: 0.3168, R2: 0.7362
age:
  MAE: 0.0486, MSE: 0.0046, RMSE: 0.0681, R2: -0.0893
x:
  MAE: 9.9034, MSE: 158.4089, RMSE: 12.5861, R2: 0.7014
y:
  MAE: 10.7824, MSE: 216.7721, RMSE: 14.7232, R2: 0.8585
tetta:
  MAE: 5.1906, MSE: 49.1691, RMSE: 7.0121, R2: 0.6521
sin_phi:
  MAE: 0.6359, MSE: 0.5014, RMSE: 0.7081, R2: -0.0046
cos_phi:
  MAE: 0.6372, MSE: 0.5012, RMSE: 0.7079, R2: -0.0006

Epoch 2/100
LR: 1.00e-03
Train Loss: 73.8877, Val Loss: 59.1593
power:
  MAE: 0.2026, MSE: 0.0769, RMSE: 0.2773, R2: 0.7978
age:
  MAE: 0.0450, MSE: 0.0039, RMSE: 0.0626, R2: 0.0771
x:
  MAE: 9.9972, MSE: 163.2894, RMSE: 12.7785, R2: 0.6922
y:
  MAE: 10.5181, MSE: 208.4581, RMSE: 14.4381, R2: 0.8639
tetta:
  MAE: 4.8669, MSE: 41.2864, RMSE: 6.4255, R2: 0.7079
sin_phi:
  MAE: 0.6361, MSE: 0.4995, RMSE: 0.7068, R2: -0.0008
cos_phi:
  MAE: 0.6372, MSE: 0.5009, RMSE: 0.7077, R2: -0.0000

Epoch 3/100
LR: 1.00

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

file_path = 'spe27p_100k_2022_correct.dat'
features, targets = load_data(file_path)
X_train, X_test, y_train, y_test = train_test_split(
    features, targets, test_size=0.2, random_state=42
)

test_dataset = TensorDataset(
    torch.FloatTensor(X_test.values).to(device),
    torch.FloatTensor(y_test.values).to(device)
)

val_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Инициализация модели и перенос на GPU
input_size = X_train.shape[1]
output_size = y_train.shape[1]
model = MultiTargetNN(input_size, output_size).to(device)
model.load_state_dict(torch.load('best_model_multihead.pth', map_location=device))
model.eval()

,primary_particle,primary_energy,n_muons,n_hadrons,first_interaction_height,energy_0,energy_1,energy_2,energy_3,energy_4,...,threshold_time_134,threshold_time_135,threshold_time_136,threshold_time_137,threshold_time_138,threshold_time_139,threshold_time_140,threshold_time_141,threshold_time_142,threshold_time_143
0,14.0,1024144.000,10006.0,815.0,2200590.750,0.000000,0.000000,0.101911,0.000000,10.222194,...,0.000000,91881.507812,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,14.0,1670368.625,14978.0,1854.0,1621342.125,0.000000,6.218760,0.000000,4.344400,0.000000,...,0.000000,0.000000,61217.339844,61166.957031,61161.582031,61212.566406,61184.625000,61236.789062,0.000000,0.00000
2,14.0,1141731.125,10802.0,725.0,1894253.375,0.000000,0.000000,0.000000,0.000000,0.344776,...,0.000000,0.000000,78678.609375,78690.390625,0.000000,0.000000,78712.031250,78717.500000,78714.734375,78708.06250
3,14.0,1102080.125,16449.0,3024.0,1576046.875,10.767822,8.262124,2.476740,13.453603,15.366792,...,52149.480469,52140.515625,52160.343750,0.000000,52162.656250,0.000000,52230.859375,52154.417969,52199.988281,52148.78125
4,14.0,1272719.750,10435.0,1372.0,2638823.500,10.822570,22.756126,28.512867,17.532497,12.286385,...,88170.554688,88115.945312,88077.148438,0.000000,88060.843750,88135.984375,88109.804688,88076.132812,88117.335938,0.00000


,power,age,x,y,tetta,sin,cos
0,4.166507,1.444716,13.572407,37.022316,37.367474,-0.188303,-0.982111
1,4.933104,1.430547,40.330677,-61.980999,28.912228,-0.703970,-0.710230
2,4.133581,1.459046,-1.046695,-63.925430,37.267426,-0.125979,0.992033
3,5.190620,1.326983,40.293152,-21.836197,3.205503,0.205486,0.978660
4,5.021614,1.336776,15.542248,10.037846,6.299241,0.959007,0.283384


MultiTargetNN(
  (shared_encoder): Sequential(
    (0): Linear(in_features=293, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
  )
  (heads): ModuleList(
    (0-6): 7 x Sequential(
      (0): Linear(in_features=32, out_features=16, bias=True)
      (1): ReLU()
      (2): Linear(in_features=16, out_features=1, bias=True)
    )
  )
)

In [53]:
import numpy as np
import torch

# Функция для вычисления RMSE
def rmse(predictions, targets):
    return np.sqrt(np.mean((predictions - targets) ** 2))

# Функция для восстановления phi из sin и cos
def recover_phi(sin_vals, cos_vals):
    phi_rad = np.arctan2(sin_vals, cos_vals)
    phi_deg = np.degrees(phi_rad)
    # Нормализация к диапазону [0, 360)
    phi_deg = np.where(phi_deg < 0, phi_deg + 360, phi_deg)
    return phi_deg

def custom_metric(a, b):
    return np.minimum(np.abs(a - b), 360 - np.abs(a - b))

# Инициализация списков для сбора предсказаний и истинных значений
all_preds = []
all_targets = []

# Инференс и сбор данных
with torch.no_grad():
    for inputs, targets in val_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        outputs = model(inputs)
        
        # Сохраняем результаты для последующего расчета метрик
        all_preds.append(outputs.cpu().numpy())
        all_targets.append(targets.cpu().numpy())

# Объединение результатов всех батчей
all_preds = np.vstack(all_preds)
all_targets = np.vstack(all_targets)

# Извлечение отдельных предсказаний и целей
pred_power = all_preds[:, 0]
pred_age = all_preds[:, 1]
pred_x = all_preds[:, 2]
pred_y = all_preds[:, 3]
pred_tetta = all_preds[:, 4]
pred_sin = all_preds[:, 5]
pred_cos = all_preds[:, 6]

true_power = all_targets[:, 0]
true_age = all_targets[:, 1]
true_x = all_targets[:, 2]
true_y = all_targets[:, 3]
true_tetta = all_targets[:, 4]
true_sin = all_targets[:, 5]
true_cos = all_targets[:, 6]

# Восстановление phi из предсказаний
pred_phi = recover_phi(pred_sin, pred_cos)
true_phi = recover_phi(true_sin, true_cos)

# Вычисление RMSE для каждой переменной
rmse_power = rmse(pred_power, true_power)
rmse_age = rmse(pred_age, true_age)
rmse_x = rmse(pred_x, true_x)
rmse_y = rmse(pred_y, true_y)
rmse_tetta = rmse(pred_tetta, true_tetta)

# Специальная обработка для phi (учет циклической природы)
phi_diff = np.abs(true_phi - pred_phi)
phi_diff = np.minimum(phi_diff, 360 - phi_diff)  # Минимальная угловая разница
rmse_phi = np.sqrt(np.mean(phi_diff ** 2))

# Вывод результатов
print(f"{'Target':<10} | {'RMSE':>10}")
print("-" * 25)
print(f"{'power':<10} | {rmse_power:>10.4f}")
print(f"{'age':<10} | {rmse_age:>10.4f}")
print(f"{'x':<10} | {rmse_x:>10.4f}")
print(f"{'y':<10} | {rmse_y:>10.4f}")
print(f"{'tetta':<10} | {rmse_tetta:>10.4f}")
print(f"{'phi':<10} | {rmse_phi:>10.4f}")

# Дополнительные метрики для phi
mae_phi = np.mean(phi_diff)
acc_5deg = np.mean(phi_diff <= 5) * 100
acc_10deg = np.mean(phi_diff <= 10) * 100

print("\nAdditional metrics for phi:")
print(f"MAE: {mae_phi:.4f} degrees")
print(f"Accuracy within 5°: {acc_5deg:.2f}%")
print(f"Accuracy within 10°: {acc_10deg:.2f}%")



Target     |       RMSE
-------------------------
power      |     0.4145
age        |     0.0620
x          |    10.2218
y          |    11.3880
tetta      |     4.0055
phi        |   104.1952

Additional metrics for phi:
MAE: 90.3283 degrees
Accuracy within 5°: 2.69%
Accuracy within 10°: 5.66%
